In [1]:
import pandas as pd
from keras.layers import CuDNNLSTM
from keras.layers import Dense
from keras.models import Sequential
from sklearn.metrics import confusion_matrix, classification_report
from keras.callbacks import EarlyStopping, ModelCheckpoint
from data_managing import get_rnn_train_test_set


def make_model(input_shape, output_dim):
    print("model dim: ", input_shape, output_dim)
    model = Sequential()
    model.add(CuDNNLSTM(64, input_shape=input_shape, batch_size=None, return_sequences=False))
    model.add(Dense(200, activation='relu'))
    model.add(Dense(output_dim, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

Using TensorFlow backend.


In [2]:

selected_labels = ['Healthy control', 'Myocardial infarction', 'Bundle branch block', 'Cardiomyopathy']
window_size = 1024
trainX, trainY, testX, testY, record_list = get_rnn_train_test_set(selected_labels, window_size)

100%|██████████| 549/549 [00:06<00:00, 81.34it/s] 
432it [06:05,  1.16it/s]
50it [00:40,  1.18it/s]


In [3]:
model = make_model((trainX.shape[1], trainX.shape[2]),
                   trainY.shape[-1])

checkpoint = ModelCheckpoint('weights_best_model', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stopping = EarlyStopping(patience=5)
callbacks_list = [checkpoint, early_stopping]

model.fit(trainX, trainY,
          validation_split=0.15,
          epochs=10000,
          batch_size=512,
          callbacks=callbacks_list,
          verbose=0)

model dim:  (1, 1024) 4

Epoch 00001: val_loss improved from inf to 0.74030, saving model to weights_best_model

Epoch 00002: val_loss improved from 0.74030 to 0.71045, saving model to weights_best_model

Epoch 00003: val_loss improved from 0.71045 to 0.68110, saving model to weights_best_model

Epoch 00004: val_loss improved from 0.68110 to 0.66062, saving model to weights_best_model

Epoch 00005: val_loss improved from 0.66062 to 0.63457, saving model to weights_best_model

Epoch 00006: val_loss improved from 0.63457 to 0.61623, saving model to weights_best_model

Epoch 00007: val_loss improved from 0.61623 to 0.58552, saving model to weights_best_model

Epoch 00008: val_loss improved from 0.58552 to 0.57397, saving model to weights_best_model

Epoch 00009: val_loss improved from 0.57397 to 0.55710, saving model to weights_best_model

Epoch 00010: val_loss improved from 0.55710 to 0.53924, saving model to weights_best_model

Epoch 00011: val_loss improved from 0.53924 to 0.52550, sav

In [9]:
testX

array([[[5.3355, 5.3365, 5.3365, ..., 5.262 , 5.261 , 5.264 ]],

       [[5.324 , 5.3255, 5.3235, ..., 5.303 , 5.3   , 5.3   ]],

       [[5.264 , 5.262 , 5.26  , ..., 4.919 , 4.925 , 4.9295]],

       ...,

       [[0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ]],

       [[0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ]],

       [[0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ]]])

In [7]:
output = model.predict_classes(testX)
print(len(record_list), len(output), len(testY.argmax(axis=1)))
summed = pd.DataFrame({'predictions': output,
                       'label': testY.argmax(axis=1)})

print(confusion_matrix(testY.argmax(axis=1), output))
print(classification_report(summed['label'], summed["predictions"]))


10923 10973 10973
[[ 699 1146   20    4]
 [ 750 7254  109   59]
 [   3  463    0    0]
 [   0  345   17  104]]
              precision    recall  f1-score   support

           0       0.48      0.37      0.42      1869
           1       0.79      0.89      0.83      8172
           2       0.00      0.00      0.00       466
           3       0.62      0.22      0.33       466

   micro avg       0.73      0.73      0.73     10973
   macro avg       0.47      0.37      0.40     10973
weighted avg       0.70      0.73      0.71     10973

